<a href="https://colab.research.google.com/github/princedede/Data-Analysis-Projects/blob/main/SentimentAnalysisHotelReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import regex

import pandas as pd
import pandas_datareader.data as pdr
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

#use the following if stopwords isnt installed
#nltk.download('stopwords')

from nltk.stem import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import string
import re
import emoji
from ydata_profiling import ProfileReport


# importing csv file

dataframe_3 = pd.read_csv('7282_1.csv')

#change reviews.text to reviews
dataframe_3 = dataframe_3.rename(columns={'reviews.text': 'reviews'})

#selecting needed columns

df=dataframe_3[["name","reviews.rating","reviews"]]

#check for missing values

check_missing_val_total=df.isnull().sum()

#assigning grade to ratings
rating_high= df[df["reviews.rating"]==5]
print("rating high:",rating_high)

rating_mid=df[df["reviews.rating"]==3]
print("rating_mid:",rating_mid)

rating_low=df[df["reviews.rating"]==1]
print("rating_low:",rating_low)

#grouping by name and finding the mean

x=(df.groupby('name')['reviews.rating'].mean())
print(x)

#plotting ratings reviews.

plt.hist(df['reviews.rating'], bins = 5)
plt.show()

# Lower casing

# Change the reviews type to string
#df['reviews'] = df.loc[:, ('reviews')].astype(str)
df['reviews'] = df['reviews'].astype(str)

#Lowercase all reviews
df['reviews']= df['reviews'].apply(lambda x: x.lower())
print(df['reviews'][2]) ## to see the difference

#check if there is any special character
alphabet = string.ascii_letters+string.punctuation
print(df.reviews.str.strip(alphabet).astype(bool).any())

extracted_emojis=[]

def extract_emojis(s):
    expe = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    #return expe.findall(s)
    return expe.sub(r'',s)

for y in df['reviews']:
    #print(str(extract_emojis(y)))
    extracted_emojis.append(str(extract_emojis(y)))

    # stop words

stop_words=stopwords.words('english')
df['extracted_emojis'] = extracted_emojis
df['extracted_emojis']= df['extracted_emojis'].apply(lambda x:x if x not in stop_words else None)
print(df['extracted_emojis'][5])

# stemming

def stemming(x):
    st = PorterStemmer()
    if x is not None:
       for word in x.split():
           st.stem(word)

df['extracted_emojis'].apply(lambda x:stemming(x))
print(df['extracted_emojis'][100])

#Function to calculate sentiment score for whole data set

def senti_sc(x):
    if x is not None:
       return TextBlob(x).sentiment

df["Sentiment_score"]= df["extracted_emojis"].apply(senti_sc)
print(df.loc[0:19,['extracted_emojis','Sentiment_score']])



ModuleNotFoundError: ignored